<a href="https://colab.research.google.com/github/nyantacos/ar_pointing_system/blob/main/test_20251114_01_Kalmanfilter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [17]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

hull_tactical_market_prediction_path = kagglehub.competition_download('hull-tactical-market-prediction')

print('Data source import complete.')


Data source import complete.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [10]:
!cd /kaggle/input/

In [ ]:
import numpy as np

def kalman_filter_1d(measurements, initial_state_estimate, initial_process_variance, process_noise_variance, measurement_noise_variance):
    """
    Implements a 1D Kalman filter for a constant velocity model.

    Args:
        measurements (list or np.array): A list of observed measurements.
        initial_state_estimate (np.array): Initial estimate of the state [position, velocity].
        initial_process_variance (np.array): Initial estimate of the process covariance matrix.
        process_noise_variance (float): Variance of the process noise.
        measurement_noise_variance (float): Variance of the measurement noise.

    Returns:
        np.array: Filtered state estimates for each time step.
    """

    num_steps = len(measurements)
    filtered_states = np.zeros((num_steps, 2))

    # State transition matrix (A): x_k = A * x_{k-1} + B * u_k + w_k
    # For a constant velocity model in 1D, A represents how position and velocity change over time.
    # Assuming dt = 1 (time step).
    dt = 1
    A = np.array([[1, dt],
                  [0, 1]])

    # Process noise covariance matrix (Q)
    # Models the uncertainty in the process itself (e.g., unexpected accelerations).
    # For simplicity, assuming noise only affects acceleration, which in turn affects position and velocity.
    Q = np.array([[(dt**4)/4, (dt**3)/2],
                  [(dt**3)/2, dt**2]]) * process_noise_variance

    # Measurement matrix (H): z_k = H * x_k + v_k
    # H relates the state to the measurement. Here, we only measure position.
    H = np.array([[1, 0]])

    # Measurement noise covariance matrix (R)
    # Models the uncertainty in the measurement sensor.
    R = np.array([[measurement_noise_variance]])

    # Initial estimates
    x_hat = initial_state_estimate # Initial state estimate [position, velocity]
    P = initial_process_variance   # Initial process covariance matrix

    for i, z in enumerate(measurements):
        # 1. Predict (Time Update)
        # Predict next state
        x_hat_minus = A @ x_hat
        # Predict next process covariance
        P_minus = A @ P @ A.T + Q

        # 2. Update (Measurement Update)
        # Calculate Kalman Gain
        K = P_minus @ H.T @ np.linalg.inv(H @ P_minus @ H.T + R)

        # Update state estimate
        x_hat = x_hat_minus + K @ (z - H @ x_hat_minus)
        # Update process covariance
        P = (np.eye(len(x_hat)) - K @ H) @ P_minus

        filtered_states[i] = x_hat

    return filtered_states

# --- Example Usage ---
# Simulate some data
np.random.seed(42)
true_position = 0
true_velocity = 1

# Generate true states and noisy measurements
num_points = 50
true_positions = []
measurements = []
measurement_noise_std = 0.5
process_noise_std = 0.1 # This will be squared for variance

for _ in range(num_points):
    true_position += true_velocity + np.random.normal(0, process_noise_std)
    true_positions.append(true_position)
    measurements.append(true_position + np.random.normal(0, measurement_noise_std))

measurements = np.array(measurements)

# Kalman Filter parameters
initial_state = np.array([measurements[0], 0]) # Initial position is first measurement, initial velocity is 0
initial_covariance = np.array([[1, 0],
                               [0, 1]]) # High uncertainty initially
process_noise_var = process_noise_std**2
measurement_noise_var = measurement_noise_std**2

# Run the Kalman filter
filtered_results = kalman_filter_1d(measurements, initial_state, initial_covariance, process_noise_var, measurement_noise_var)

# Print some results
print("Original Measurements (first 5):", measurements[:5])
print("Filtered Positions (first 5):", filtered_results[:5, 0])
print("True Positions (first 5):", np.array(true_positions)[:5])

# You can also plot these results to visualize the filtering effect
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(true_positions, label='True Position', linestyle='--')
plt.plot(measurements, label='Measurements', marker='.', alpha=0.6)
plt.plot(filtered_results[:, 0], label='Kalman Filtered Position')
plt.xlabel('Time Step')
plt.ylabel('Position')
plt.title('1D Kalman Filter Example')
plt.legend()
plt.grid(True)
plt.show()

In [12]:
%cd /kaggle/input

/kaggle/input


!ls

In [16]:
!ls -al

total 8
drwxr-xr-x 2 root root 4096 Nov 14 09:27 .
drwxr-xr-x 3 root root 4096 Nov 14 09:27 ..
